In [20]:
import psycopg2
from psycopg2 import OperationalError
from psycopg2 import sql
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy.optimize import curve_fit
import seaborn as sns
from IPython.display import HTML
import webbrowser
from collections import defaultdict
import networkx as nx
import csv
import os
from datetime import datetime, timedelta

os.chdir('/home/francisco/MECAD/2º Ano/Estágio/forcera/scripts/scripts_py')


# LIGAÇÃO À BASE DE DADOS
conn = psycopg2.connect(
    host = "contratos-base-gov1.cf87yxnqgph8.eu-central-1.rds.amazonaws.com",
    port = 5432,
    #database = "contratosbasegov",
    user = "contratosbasegov",
    password = "8n9nyeTBFUyCcLJShNrZdPUai2KQkue4")
cur = conn.cursor()

In [30]:
cur.execute('''ROLLBACK;''')

***

**1.** Printar ids dos concursos públicos entre os dias 1 e 4 de janeiro de 2024

***

In [21]:
dias = np.arange(1,3,1, dtype = int)

for i in dias:
    cur = conn.cursor()
    cur.execute('''
                SELECT id
                FROM contratos_basegov
                WHERE data_publicacao = '2024-01-%s' AND tipo_procedimento = 'Concurso público'
                ;''', (int(i),))
    
    dt = cur.fetchall()
    print(dt, '\n\n')

[(10424267,), (10424266,), (10424264,), (10424263,), (10424262,)] 


[(10424689,), (10424672,), (10424653,), (10424639,), (10424328,), (10424326,), (10424324,), (10424321,), (10424320,), (10424316,), (10424315,), (10424314,), (10424306,), (10424304,)] 




***

**2.** Printar dia de hoje e ontem no formato ANO-MES-DIA

***

In [22]:
hoje = datetime.today().strftime('%Y-%m-%d')

yesterday = datetime.now() - timedelta(1)
ontem = datetime.strftime(yesterday, '%Y-%m-%d')

print(hoje, ontem)

2024-02-13 2024-02-12


***

**3.** Printar ids dos concursos públicos de ontem

***

In [23]:
cur = conn.cursor()
cur.execute('''
            SELECT id
            FROM contratos_basegov
            WHERE data_publicacao = %s AND tipo_procedimento = 'Concurso público'
            ;''', (ontem,))

dt = cur.fetchall()
print(dt, '\n\n')

[(10555963,), (10555962,), (10555961,), (10555960,), (10555957,), (10555954,), (10555926,), (10555925,), (10555921,), (10555917,), (10555916,), (10555913,), (10555912,), (10555900,), (10555873,), (10555867,), (10555847,), (10555842,), (10555800,), (10555782,), (10555721,), (10555666,), (10555661,), (10555644,), (10555642,), (10555609,), (10555598,), (10555587,), (10555549,), (10555476,), (10555398,), (10555372,), (10555364,), (10555335,), (10555313,), (10555294,), (10555279,), (10555239,), (10555225,), (10555214,), (10555170,), (10555106,), (10555094,), (10555083,), (10555033,), (10555025,), (10555022,), (10554959,), (10554950,), (10554947,), (10554944,), (10554932,), (10554924,), (10554907,), (10554900,), (10554831,), (10554824,), (10554792,), (10554708,), (10554703,), (10554697,), (10554681,), (10554670,), (10554661,), (10554630,), (10554556,), (10554516,), (10554502,), (10554478,), (10554474,), (10554444,), (10554438,), (10554333,), (10554302,), (10554291,), (10554232,), (10554287,)

***

**4.** Printar ids dos concursos públicos de ontem que haja apenas uma entidade concorrente

***

In [24]:
cur = conn.cursor()
cur.execute('''
            SELECT contratos_basegov."id"
            FROM contratos_basegov
            JOIN concursos_publicos ON contratos_basegov."id" = concursos_publicos."id"
            WHERE data_publicacao = %s AND tipo_procedimento = 'Concurso público' AND concursos_publicos."nr_entidadesconcorrentes" = 1
            ''', (ontem,))

d = cur.fetchall()
print(d, '\n\n')

[] 




***

**5.** Escrever **IDs** e **data de publicacao** dos concursos públicos de ontem na tabela *table_temp*

***

In [34]:
cur = conn.cursor()
cur.execute('''
            SELECT id
            FROM contratos_basegov
            WHERE data_publicacao = %s AND tipo_procedimento = 'Concurso público'
            ;''', (ontem,))

#t = [item[0] for item in cur.fetchall()]
t = cur.fetchall()
print(t, '\n\n')

[(10555963,), (10555962,), (10555961,), (10555960,), (10555957,), (10555954,), (10555926,), (10555925,), (10555921,), (10555917,), (10555916,), (10555913,), (10555912,), (10555900,), (10555873,), (10555867,), (10555847,), (10555842,), (10555800,), (10555782,), (10555721,), (10555666,), (10555661,), (10555644,), (10555642,), (10555609,), (10555598,), (10555587,), (10555549,), (10555476,), (10555398,), (10555372,), (10555364,), (10555335,), (10555313,), (10555294,), (10555279,), (10555239,), (10555225,), (10555214,), (10555170,), (10555106,), (10555094,), (10555083,), (10555033,), (10555025,), (10555022,), (10554959,), (10554950,), (10554947,), (10554944,), (10554932,), (10554924,), (10554907,), (10554900,), (10554831,), (10554824,), (10554792,), (10554708,), (10554703,), (10554697,), (10554681,), (10554670,), (10554661,), (10554630,), (10554556,), (10554516,), (10554502,), (10554478,), (10554474,), (10554444,), (10554438,), (10554333,), (10554302,), (10554291,), (10554232,), (10554287,)

In [35]:
cur = conn.cursor()

for i in t:
    cur.execute('''INSERT INTO table_temp(id, data) VALUES (%s, %s);''', (i,ontem)) 

conn.commit() 